In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
# context = Context('amended-3-less-features-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=9)
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=7)
# context = Context('amended-3-less-features-no-anomaly-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=7)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

# Hyper parameter tune

In [ ]:
# 1
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.4,
    'weight_p': 0.5,
    'drops': [0.4] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}
hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, len(context.dep_var)]
learner = get_new_model(context, databunch, hyper_params)
learner.fit(60, lr=1e-1)

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.15,
    'input_p': 0.2,
    'weight_p': 0.25,
    'drops': [0.2] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}
hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, len(context.dep_var)]
learner = get_new_model(context, databunch, hyper_params)
learner.fit(60, lr=1e-1)

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.075,
    'input_p': 0.1,
    'weight_p': 0.125,
    'drops': [0.1] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}
hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, len(context.dep_var)]
learner = get_new_model(context, databunch, hyper_params)
learner.fit(60, lr=1e-1)

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0,
    'input_p': 0,
    'weight_p': 0,
    'drops': [0.0] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': MSELossFlat()
}
hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, len(context.dep_var)]
learner = get_new_model(context, databunch, hyper_params)
learner.fit(60, lr=1e-1)

In [ ]:
learner.fit(10, lr=1e-1)

In [ ]:
xb_valid, yb_valid = ni(databunch.valid_dl)
yb_valid = to_np(yb_valid)

In [ ]:
yb_cont_valid_denorm = denormalize(yb_valid[:,:2], *context.stat_y)
pb_valid = denormalize(pb_valid[:,:2], *context.stat_y)

In [ ]:
model = learner.model
model.eval()
model.reset()
pb_valid = model(*xb_valid)[0]
pb_valid = to_np(pb_valid)

In [ ]:
# mre
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

i = 0
print('mre', (np.abs(pb_valid[:,i] - yb_cont_valid_denorm[:,i]) / yb_cont_valid_denorm[:,i]).mean())
print('rmse', sqrt(mean_squared_error(pb_valid[:,i], yb_cont_valid_denorm[:,i])))
print('r2_score', r2_score(pb_valid[:,i], yb_cont_valid_denorm[:,i]))

In [ ]:
# Now let's quickly train a random forest with various kinds of datasets
from sklearn.ensemble import forest
from sklearn.ensemble import RandomForestRegressor
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
xb, yb = ni(databunch.train_dl)
X_train, y_train = to_np(xb[1]), to_np(yb)

xb, yb = ni(databunch.valid_dl)
X_valid, y_valid = to_np(xb[1]), to_np(yb)
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3, max_features=0.5)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores